## Remove NaN From Preprocessed Enron Datasets

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("preprocessed_enron.csv")
#Remove NaN value
for i, msg in enumerate(df['body']):
    # print(i, msg)
    if msg is np.nan:
        df = df.drop(i)
for i, msg in enumerate(df['body']):
    if msg is np.nan:
        print(i, msg)
df = df.reset_index(drop=True)

# 3. allow user to select reference corpus essential

#Feature3

## User input for Chosing Dataset

In [2]:
while True:
    try:
        dataset_num = input('1: Enron+K1 Datasets, 2: Enron+K2 Datasets\nChoose 1 or 2:')
        dataset_num = int(dataset_num)
        if dataset_num == 1 or dataset_num==2:
            break
    except EOFError as e:
        print(e)
        break
    dataset_num = input('1: Enron+K1 Datasets, 2: Enron+K2 Datasets\nChoose 1 or 2:')
    dataset_num = dataset_num.strip() # 空白文字のみの入力は無視する
    if dataset_num == 1 or dataset_num==2:
        break

## Merge Enron Dataset with the Dataset User Chose

In [3]:
if(dataset_num == 1):
    f = open('K1_dataset.txt', 'r', encoding='UTF-8')
else:
    f = open('K2_dataset.txt', 'r', encoding='UTF-8')
K1_data = f.read()
df.loc[len(df)] = ['K1@dataset.com', K1_data] #add K1 datasets into the last row
f.close()

# 1. count, list and order the frequency of words

#Feature1

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
documents = df['body'].tolist()

# for i, msg in enumerate(df['body']):
#     # print(i, msg)
#     if msg is np.nan:
#         print(i, msg)

#tfidf_vectorizer = TfidfVectorizer()
#tfidf_vectors = tfidf_vectorizer.fit_transform(documents) # keyword frequency list
tf_vectorizer = CountVectorizer()
tf_vectors = tf_vectorizer.fit_transform(documents)         # word frequency list
# del tf_vectorizer
# del tfidf_vectorizer# データ分割r

## Display Words and Freq

In [5]:
df_disp_word_freq = pd.DataFrame.from_dict(data = tf_vectorizer.vocabulary_, orient="index", columns=["frequency"]).sort_values('frequency', ascending=False)
df_disp_word_freq

,frequency
zoning,9396
zones,9395
zone,9394
znamenny,9393
zivic,9392
...,...
005,4
000mmbtu,3
000f749c00763c4c3b9aca0044810ab2,2
000,1


## Make Words List Corresponding with the Index of "tf_vectors"

### "tf_vectors" represents frequency word matrix, 
Columns of which are all words in all documents, 

Rows of which are the value of furequency words in a document.

In [6]:
# Make words dictionary and it is used in Train and Test cases
words=tf_vectorizer.get_feature_names_out()
#print(words[5020:5025])

## tf_vectors matrix

In [7]:
#tfidf_mat = tfidf_vectors.toarray() # dead every time
#del tfidf_vectors
tf_mat = tf_vectors.toarray()
del tf_vectors
df['tf'] = tf_mat.tolist()
#df['tfidf'] = tfidf_mat.tolist()

## Remove Zero Vector Record for Normalizing Token Frequency

In [8]:
for i, vec in enumerate(df['tf']):
    if sum(vec) == 0:
        df = df.drop(i)

df = df.reset_index(drop=True)

## Make Normalirzed Token Frequency List into df

In [9]:
# Normalization
# we generaly name 'ntf' for normalized term frequency
normalized_tf_list = []
for row in df['tf']:
    num_words = sum(row)
    normalized_tf = []
    for x in row:
        normalized_tf.append(x/num_words)

    normalized_tf_list.append(normalized_tf)
df['ntf'] = normalized_tf_list

#Tf-idf の代わりに利用する keyness を作る

ここでは　df['keyness'] を作成し追加したい

In [10]:
import math
# we generaly name 'ntf' for normalized term frequency

# First, create the shared normalized tf vector
shared_ntf = None # shared ntf of all document

matrix = []
for row in df['ntf']:
    matrix.append(row)

np_matrix = np.array(matrix)

mean_vector = np_matrix.mean(axis=0)

shared_ntf = mean_vector.tolist()

def keyness(ntf_vector1, ref_ntf_vector2): # freq_vector1 and freq_vector2 are both already normalized
    keyness_vec = []
    for i, x in enumerate(ntf_vector1):
        if ntf_vector1[i] == 0:
            keyness_vec.append(0)
        else:
            keyness_vec.append(math.log2(ntf_vector1[i]/ref_ntf_vector2[i]))

    return keyness_vec

keyness_mat = []
for ntf_vector in df['ntf']:
    keyness_vec = keyness(ntf_vector, shared_ntf)
    keyness_mat.append(keyness_vec)

# Add Keyness value matrix into df
df['keyness'] = keyness_mat

In [11]:
#確認用
i = 22
msg = df['body'][i]
max_value = max(keyness_mat[i])
max_idx = keyness_mat[i].index(max_value)
print(words[max_idx])

print(msg)
#print(df['author'][i])


mechelle
Mechelle,
            I will let you make the call, MW's or $'s. Look at the spreadsheet  below and give me your thoughts.
 
                                                              Chris
              




## Split data into Train Data and Test Data

In [12]:
from sklearn.model_selection import train_test_split
X_keyness_train, X_keyness_test, Y_keyness_train, Y_keyness_test = train_test_split(df['keyness'],df['author'],test_size=0.2,shuffle=True)
X_tf_train, X_tf_test, Y_tf_train, Y_tf_test = train_test_split(df['tf'],df['author'],test_size=0.2,shuffle=True)

## Check the number of the Authors

In [13]:
# How many author?
authors = set(Y_keyness_test)
authors_list = [author for author in authors]

## Create Reference_vectors
size: the number of author

This is made of Train data

In [14]:
# df_X = pd.DataFrame(X_tf_train.values.tolist())
# df_Y = pd.DataFrame(Y_tf_train.values.tolist())


df_train = pd.concat((X_keyness_train, Y_keyness_train.rename('author')), axis=1)
#
reference_vectors = {}
for author in authors:

    df_author = df_train.groupby('author').get_group(author)

    matrix = []
    for row in df_author['keyness']:
        matrix.append(row)

    np_matrix = np.array(matrix)

    mean_vector = np_matrix.mean(axis=0)
    reference_vectors[author] = mean_vector.tolist()

In [15]:
for author, ref_vec in reference_vectors.items():
    max_value = max(ref_vec)
    max_idx =ref_vec.index(max_value)
    print(f'{author}: {words[max_idx]}')


andy.zipper@enron.com: andy
ben.jacoby@enron.com: ben
chris.stokley@enron.com: chris
v.weldon@enron.com: charlie
j..kean@enron.com: see
hunter.shively@enron.com: hunter


In [16]:
authors

{'andy.zipper@enron.com',
 'ben.jacoby@enron.com',
 'chris.stokley@enron.com',
 'hunter.shively@enron.com',
 'j..kean@enron.com',
 'v.weldon@enron.com'}

In [17]:
# import nltk
# from nltk.corpus import stopwords

# nltk.download('stopwords')
# stop_words = stopwords.words('english')

In [18]:
# start からend までのwordの配列を返す
def extract_features_words(freq_vector, words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        max_value = max(setX)
        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)

        ### if exclude stopwords
        # if max_word not in stop_words:
        #     if count>= start:
        #         result.append(max_index)
        #     count+=1

        if count>= start:
            result.append(max_word)
        count += 1


    return result

In [19]:
# start からend までのword IDの配列を返す
def extract_features(freq_vector, words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        try:
            max_value = max(setX)
        except ValueError:
            return result

        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)

        ### if exclude stopwords
        # if max_word not in stop_words:
        #     if count>= start:
        #         result.append(max_index)
        #     count+=1

        if count>= start:
            result.append(max_word)
        count += 1

        #


    return result

In [20]:
authors

{'andy.zipper@enron.com',
 'ben.jacoby@enron.com',
 'chris.stokley@enron.com',
 'hunter.shively@enron.com',
 'j..kean@enron.com',
 'v.weldon@enron.com'}

In [21]:
vec = reference_vectors['j..kean@enron.com']
top_words = extract_features_words(vec, words=words,  start=0, end=150)
print(top_words)


['see', 'attached', 'ken', 'we', 'please', 'my', 'of', 'our', 'but', 'think', 'for', 'have', 'be', 'as', 'and', 'some', 'is', 'with', 'this', 'about', 'he', 'california', 'the', 'blackberry', 'to', 'like', 'make', 'management', 'message', 'handheld', 'net', 'organizations', 'wireless', 'should', 'organization', 'from', 'are', 'it', 'fyi', 'not', 'rto', 'would', 'corporate', 'synchronizing', 'lists', 'humor', 'sent', 'was', 'his', 'in', 'so', 'buydown', 'crisis', 'communications', 'cftc', 'folder', 'press', 'india', 'steven', 'kean', 'council', 'me', 'what', 'don', 'very', 'much', 'or', 'been', 'issues', 'that', 'associations', 'profits', 'at', 'objectives', 'contacts', 'reg', 'campaign', 'you', 'advertising', 'inbox', 'presentations', 'forms', 'people', 'know', 'they', 'more', 'committee', 'meeting', 'personal', 'business', 'your', 'federal', 'revised', 'credit', 'way', 'leadership', 'brazil', 'offline', 'fine', 'security', 'mexico', 'one', 'japan', 'www', 'sec', 'will', 'him', 'better

In [22]:
def get_similarity(feature_vector1,feature_vector2):
    return len(set(feature_vector1) & set(feature_vector2))



In [23]:

INF = float('inf')

def predict(questioned_vector,reference_vectors):
    start = 0
    end = 20
    suspected = [author for author in authors]
    innocent_list = []
    while(len(suspected) > 1):
        Q_features = extract_features(questioned_vector, words, start, end)
        similarityWithQ = {}

        for author, reference_vector in reference_vectors.items():
            if author in suspected: #
                feature_vector = extract_features(reference_vector, words, start, end)
                score = get_similarity(feature_vector,Q_features)
                similarityWithQ[author]=score

        # innocent_list に含まれない著者の中から1人を選ぶ

        innocent = min(similarityWithQ, key=similarityWithQ.get)
        #print(innocent)
        # print(type(innocent))
        # print(f'{min(similarityWithQ, key=similarityWithQ.get)} may be innocent.')
        suspected.remove(innocent)
        # similarityWithQ[innocent] = INF



        end += 20
    return suspected[0]

In [24]:
# suspected = [author for author in authors]
# suspected.remove('andy.zipper@enron.com')
# suspected

## Check function in Train Data

In [25]:
i = 0
bad_guy = predict(df['keyness'][i], reference_vectors)
print(f'bad guy : {bad_guy}')
print('-'*20)
print(f"True author : {df['author'][i]}")

bad guy : v.weldon@enron.com
--------------------
True author : chris.stokley@enron.com


## Check function in Test Data

In [26]:
X_keyness_test

2392    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1593    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
110     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
63      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
1016    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1846    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1483    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1351    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1102    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: keyness, Length: 531, dtype: object

In [27]:
all_test_data = len(X_keyness_test)

In [28]:
match_cnt = 0
all_test_data = len(X_keyness_train)
for i in X_keyness_train.index:
    bad_guy = predict(df['keyness'][i], reference_vectors)
    if df['author'][i] == bad_guy:
        match_cnt = match_cnt + 1

print(f'Math rate is: {match_cnt/all_test_data*100} % ')
    #print(f'bad guy : {bad_guy}')
    #print('-'*20)
    #print(f"True author : {df['author'][i]}")

In [ ]:
# def predict(questioned_vector, reference_vectors):
#     suspected = [author for author in authors]

#     comparedSize = 20
#     while(len(suspected) > 1):




